In [7]:
import numpy as np
import nibabel as nib
import os
import glob
from skimage.exposure import rescale_intensity
from scipy.ndimage import morphology
import SimpleITK as sitk

In [2]:
def match_histo(): 
    template = glob.glob(os.path.join('.',  "Data", 'Train','img','IBSR_06.nii'))
    images = glob.glob(os.path.join('.', "Data", '*','img','*'))
    output = glob.glob(os.path.join('.', "Data_processed"))
    
    template_nii = nib.load(template[0])
    template_data = template_nii.get_fdata().astype('float32')
    template_data = np.squeeze(template_data)
    for image in images:
        img_nii = nib.load(image)
        affine=img_nii.affine
        img = img_nii.get_fdata().astype('float32')
        img = np.squeeze(img)
    
        img_matched = histmatch(img,template_data)
        img_matched = nib.Nifti1Image(img_matched, affine= affine)
        print(os.path.basename(image) )
        print(output[0])
        print(os.path.join(output[0],os.path.basename(image)))
        nib.save(img_matched, os.path.join(output[0],os.path.basename(image)))

In [3]:
def Adativehistogram_equalization(ref,alpha,beta):

    histogramEqualization = sitk.AdaptiveHistogramEqualizationImageFilter()
    
    histogramEqualization.SetAlpha(alpha)
    histogramEqualization.SetBeta(beta)
    equalized_volume = histogramEqualization.Execute(ref)
    
    
    return equalized_volume

In [8]:
#Adaptive Histogram Equalization for reference Image

maskPath = './Data/Train/seg/IBSR_06.nii'
refImgPath = './Data/Train/img/IBSR_06.nii'
masknib = nib.load(maskPath)
maskData = masknib.get_fdata()
maskData = np.squeeze(maskData)
maskData[maskData==2]=1
maskData[maskData==3]=1
labels=maskData
labels= labels.astype(bool)
allLabels = np.empty(shape=[masknib.shape[0],masknib.shape[1],masknib.shape[2]], dtype='bool')

for i in range(masknib.shape[2]):
    allLabels[:,:,i]=morphology.binary_fill_holes(labels[:,:,i],structure=np.ones((5,5)))
allLabels= allLabels.astype(int)
maskData=allLabels

imgnib =nib.load(refImgPath)
affine=imgnib.affine
itkimage = sitk.ReadImage(refImgPath)
volume = Adativehistogram_equalization(itkimage,alpha=0.5,beta=0.5)
arrayImage = sitk.GetArrayFromImage(volume)
arraynp = np.array(arrayImage)
arraynp=np.swapaxes(arraynp, 0, 2)


finalResult = np.multiply(maskData,arraynp)
image=finalResult
rescalefinalResult = rescale_intensity(image, in_range='image', out_range=(0,255))
#result=np.swapaxes(finalResult, 0, 2)

img = nib.Nifti1Image(rescalefinalResult, affine=affine)
volName = 'AHE_IBSR_06.nii'
nib.save(img, os.path.join('./', volName))


In [60]:

def histmatch(im_ref,im_t):
    n_bins = 256
    
    ref_hist,bins = np.histogram( im_ref.flatten(), n_bins, density = True )
    t_hist,bins = np.histogram( im_t.flatten(), n_bins, density = True )
    
    cdf_ref = ref_hist.cumsum()
    cdf_t = t_hist.cumsum()
    cdf_ref = (255 * cdf_ref / cdf_ref[-1]).astype(np.uint8) #normalize
    cdf_t = (255 * cdf_t / cdf_t[-1]).astype(np.uint8) #normalize
    
    im_tmp = np.interp( im_ref.flatten(), bins[:-1],cdf_ref)
    im_ret = np.interp( im_tmp, cdf_t, bins[:-1])
    
    im_ret = im_ret.reshape( im_ref.shape)
    return im_ret


In [61]:
match_histo()

IBSR_02.nii
.\Data_processed
.\Data_processed\IBSR_02.nii
IBSR_10.nii
.\Data_processed
.\Data_processed\IBSR_10.nii
IBSR_15.nii
.\Data_processed
.\Data_processed\IBSR_15.nii
IBSR_01.nii
.\Data_processed
.\Data_processed\IBSR_01.nii
IBSR_03.nii
.\Data_processed
.\Data_processed\IBSR_03.nii
IBSR_04.nii
.\Data_processed
.\Data_processed\IBSR_04.nii
IBSR_05.nii
.\Data_processed
.\Data_processed\IBSR_05.nii
IBSR_06.nii
.\Data_processed
.\Data_processed\IBSR_06.nii
IBSR_07.nii
.\Data_processed
.\Data_processed\IBSR_07.nii
IBSR_08.nii
.\Data_processed
.\Data_processed\IBSR_08.nii
IBSR_09.nii
.\Data_processed
.\Data_processed\IBSR_09.nii
IBSR_16.nii
.\Data_processed
.\Data_processed\IBSR_16.nii
IBSR_18.nii
.\Data_processed
.\Data_processed\IBSR_18.nii
IBSR_11.nii
.\Data_processed
.\Data_processed\IBSR_11.nii
IBSR_12.nii
.\Data_processed
.\Data_processed\IBSR_12.nii
IBSR_13.nii
.\Data_processed
.\Data_processed\IBSR_13.nii
IBSR_14.nii
.\Data_processed
.\Data_processed\IBSR_14.nii
IBSR_17.nii
.\

In [13]:
template = glob.glob(os.path.join('.','template.nii' ))

In [20]:
os.path.join(output,os.path.basename(image))

NameError: name 'image' is not defined

In [47]:
out_range=(0,255)
out_range


(0, 255)